# The Contextual Bandits

In [1]:
import tensorflow.compat.v1 as tf
import tf_slim as slim
import numpy as np
tf.compat.v1.disable_eager_execution()  # eager execution 관련 에러 해결

### The Contextual Bandits
- 콘텍스트 밴딧 정의
- 손잡이가 4개인 3개의 밴딧을 이용
- 각각의 밴딧은 각각의 손잡이에 대해 다른 성공 확률 가짐
- 최고의 결과 얻기 위해서는 다른 액션이 요구됨

In [2]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        # 밴딧들의 손잡이 목록 작성. 각 밴딧은 각각 손잡이 4, 2, 1이 최적
        self.bandits = np.array([[0.2, 0, -0.0, -5], [0.1, -5, 1, 0.25], [-5, 5, 5, 5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        # 각각의 에피소드에 대해 랜덤한 상태를 반환
        self.state = np.random.randint(0, len(self.bandits)) 
        return self.state
    
    def pullArm(self, action):
        # 랜덤한 수를 얻음
        bandit = self.bandits[self.state, action]
        result = np.random.randn(1)
        if result > bandit:
            # 양의 보상 반환
            return 1
        else:
            # 음의 보상 반환
            return -1

### The Policy-Based Agent
- 입력으로 현재의 상태를 받아 액션을 반환
- 에이전트는 환경의 상태를 조건으로 삼아 액션을 취함
- 에이전트는 1개의 가중치 세트를 이용
- 각 가중치의 값 : 주어진 밴딧의 특정 손잡이를 선택할 때 반환되는 값의 추정값

In [3]:
class agent():
    def __init__(self, lr, s_size, a_size):
        # 네트워크의 피드포워드 부분. 에이전트는 상태를 받아서 액션 출력
        self.state_in = tf.placeholder(shape=[1], dtype=tf.int32)
        state_in_OH = slim.one_hot_encoding(self.state_in, s_size)
        output = slim.fully_connected(state_in_OH, a_size,
                                     biases_initializer=None, activation_fn=tf.nn.sigmoid,
                                      weights_initializer=tf.ones_initializer())
        self.output = tf.reshape(output, [-1])
        self.chosen_action = tf.argmax(self.output, 0)
        
        # 학습 과정을 구현한다.
        # 비용을 계산하기 위해 보상과 선택된 액션을 네트워크에 피드하고,
        # 네트워크를 업데이트하는 데에 이를 이용한다. 
        
        self.reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)  # 선택한 손잡이의 보상
        self.action_holder = tf.placeholder(shape=[1], dtype=tf.int32)  # 선택할 손잡이
        self.responsible_weight = tf.slice(self.output, self.action_holder, [1])
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)   # 손실함수 최소화


### Training the Agent
- 에이전트는 환경의 상태를 알아내고, 액션을 취하고, 보상을 받음으로써 학습함

In [4]:
tf.reset_default_graph()  # 텐서플로 그래프 리셋

# 밴딧 로드
cBandit = contextual_bandit() 
# 에이전트 로드
myAgent = agent(lr=0.001, s_size=cBandit.num_bandits, a_size=cBandit.num_actions)  
# 네트워크 내부를 들여다보기 위해 평가할 가중치
weights = tf.trainable_variables()[0]

# 에이전트를 학습시킬 전체 에피소드 수 설정
total_episodes = 10000
# 밴딧에 대한 점수판을 0으로 설정
total_reward = np.zeros([cBandit.num_bandits, cBandit.num_actions])
# 랜덤한 액션을 취할 가능성 결정 (exploration, exploitation)
e = 0.1

init = tf.global_variables_initializer()

#텐서플로 그래프 론칭
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        # 환경으로부터 상태 가져오기
        s = cBandit.getBandit() 
        
        if np.random.rand(1) < e:
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action, feed_dict={myAgent.state_in:[s]})
            
        # 주어진 밴딧에 대해 액션을 취한 데 대한 보상을 얻는다.
        reward = cBandit.pullArm(action)
        
        # 네트워크를 업데이트한다.
        feed_dict = {myAgent.reward_holder:[reward],
                    myAgent.action_holder:[action],
                    myAgent.state_in:[s]}
        _,ww = sess.run([myAgent.update,weights], feed_dict=feed_dict)
        
        # 보상의 총계 업데이트
        total_reward[s, action] += reward
        if i%500 == 0:
            print("Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward,axis=1)))
        i += 1
        
for a in range(cBandit.num_bandits):
    print("\nThe agent thinks action " + str(np.argmax(ww[a])+1) + " for bandit " + str(a+1) + " is the most promising....")
    if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
        print("...and it was right!")
    else:
        print("...and it was wrong!")

Instructions for updating:
Use `tf.cast` instead.
Mean reward for each of the 3 bandits: [0.   0.25 0.  ]


C:\Users\syj47\anaconda3\envs\tensorflowRL\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Mean reward for each of the 3 bandits: [39.75 29.5  37.5 ]
Mean reward for each of the 3 bandits: [76.5  66.75 75.  ]
Mean reward for each of the 3 bandits: [115.   104.5  110.25]
Mean reward for each of the 3 bandits: [148.5  145.25 145.5 ]
Mean reward for each of the 3 bandits: [186.75 184.   180.5 ]
Mean reward for each of the 3 bandits: [228.25 217.5  217.  ]
Mean reward for each of the 3 bandits: [265.25 257.25 251.75]
Mean reward for each of the 3 bandits: [299.25 293.75 289.25]
Mean reward for each of the 3 bandits: [343.   331.   320.75]
Mean reward for each of the 3 bandits: [380.5  370.5  355.75]
Mean reward for each of the 3 bandits: [418.   413.5  389.25]
Mean reward for each of the 3 bandits: [458.25 450.   426.5 ]
Mean reward for each of the 3 bandits: [498.5  486.5  461.25]
Mean reward for each of the 3 bandits: [532.25 524.5  499.  ]
Mean reward for each of the 3 bandits: [573.   561.75 529.5 ]
Mean reward for each of the 3 bandits: [609.75 602.25 568.25]
Mean reward fo